In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler


In [37]:
df = pd.read_csv('./framingham.csv', index_col=False)
df

,Unnamed: 0,sex,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,0,male,39,4.0,No,0.0,No,No,No,No,195.0,106.0,70.0,26.97,80.0,77.0,0
1,1,female,46,2.0,No,0.0,No,No,No,No,250.0,121.0,81.0,28.73,95.0,76.0,0
2,2,male,48,1.0,Yes,20.0,No,No,No,No,245.0,127.5,80.0,25.34,75.0,70.0,0
3,3,female,61,3.0,Yes,30.0,No,No,Yes,No,225.0,150.0,95.0,28.58,65.0,103.0,1
4,4,female,46,3.0,Yes,23.0,No,No,No,No,285.0,130.0,84.0,23.10,85.0,85.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4233,4233,male,50,1.0,Yes,1.0,No,No,Yes,No,313.0,179.0,92.0,25.97,66.0,86.0,1
4234,4234,male,51,3.0,Yes,43.0,No,No,No,No,207.0,126.5,80.0,19.71,65.0,68.0,0
4235,4235,female,48,2.0,Yes,20.0,NaN,No,No,No,248.0,131.0,72.0,22.00,84.0,86.0,0
4236,4236,female,44,1.0,Yes,15.0,No,No,No,No,210.0,126.5,87.0,19.16,86.0,NaN,0


Let's drop the unnamed:0 column as we don't need it

**Description of Variables**
- sex - male or female
- age - Age of the patient
- education
- currentsmoker - whether or not the patient is a current smoker
- cigsperday - the number of cigarettes that the person smoked on average in one day
- BPMeds - whether or not the patient was on blood pressure medication
- PrevalentStroke - whether or not the patient had previously had a stroke
- PrevalentHyp - whether or not the patient was hypertensive 
- diabetes - whether or not the patient had diabetes
- TotChol - total cholesterol level
- SysBP - systolic blood pressure
- diaBP - diastolic blood pressure
- BMI - Body Mass Index
- heartrate - heart rate
- glucose - glucose level
- TenYearCHD - 10 year risk of coronary heart disease CHD (“1”, means “Yes”, “0” means “No”)

## DATA EXPLORATION

Let's drop the unnamed column as we don't need it in our dataset

In [38]:
df.drop(columns=['Unnamed: 0'], inplace=True)

In [39]:
df.shape

(4238, 16)

There are 4238 observations and 16 features

In [40]:
cols = df.columns
cols

Index(['sex', 'age', 'education', 'currentSmoker', 'cigsPerDay', 'BPMeds',
       'prevalentStroke', 'prevalentHyp', 'diabetes', 'totChol', 'sysBP',
       'diaBP', 'BMI', 'heartRate', 'glucose', 'TenYearCHD'],
      dtype='object')

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4238 entries, 0 to 4237
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   sex              4238 non-null   object 
 1   age              4238 non-null   int64  
 2   education        4133 non-null   float64
 3   currentSmoker    4238 non-null   object 
 4   cigsPerDay       4209 non-null   float64
 5   BPMeds           4185 non-null   object 
 6   prevalentStroke  4238 non-null   object 
 7   prevalentHyp     4238 non-null   object 
 8   diabetes         4238 non-null   object 
 9   totChol          4188 non-null   float64
 10  sysBP            4238 non-null   float64
 11  diaBP            4238 non-null   float64
 12  BMI              4219 non-null   float64
 13  heartRate        4237 non-null   float64
 14  glucose          3850 non-null   float64
 15  TenYearCHD       4238 non-null   int64  
dtypes: float64(8), int64(2), object(6)
memory usage: 529.9+ KB


In [42]:
df.describe()

,age,education,cigsPerDay,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
count,4238.000000,4133.000000,4209.000000,4188.000000,4238.000000,4238.000000,4219.000000,4237.000000,3850.000000,4238.000000
mean,49.584946,1.978950,9.003089,236.721585,132.352407,82.893464,25.802008,75.878924,81.966753,0.151958
std,8.572160,1.019791,11.920094,44.590334,22.038097,11.910850,4.080111,12.026596,23.959998,0.359023
min,32.000000,1.000000,0.000000,107.000000,83.500000,48.000000,15.540000,44.000000,40.000000,0.000000
25%,42.000000,1.000000,0.000000,206.000000,117.000000,75.000000,23.070000,68.000000,71.000000,0.000000
50%,49.000000,2.000000,0.000000,234.000000,128.000000,82.000000,25.400000,75.000000,78.000000,0.000000
75%,56.000000,3.000000,20.000000,263.000000,144.000000,89.875000,28.040000,83.000000,87.000000,0.000000
max,70.000000,4.000000,70.000000,696.000000,295.000000,142.500000,56.800000,143.000000,394.000000,1.000000


**Checking the categorical and numerical columns in the dataset**

In [44]:
cats_cols = df.select_dtypes(include='object').columns.tolist()
num_cols = df.select_dtypes(include=np.number).columns.tolist()
cats_cols

['sex',
 'currentSmoker',
 'BPMeds',
 'prevalentStroke',
 'prevalentHyp',
 'diabetes']

As we can see, we have 6 categorical features(sex, currentSmoker, BPMeds, prevalentStroke, prevalentHyp, and diabetes) while the rest are all numerical features.


Let's explore the categorical columns

In [46]:
print(f"{'feature': <20} \t {'no. of values': <20}")
print('-'*40)
for col in cats_cols:
    print(f"{col: <20} \t {df[col].nunique(): <20}")

feature              	 no. of values       
----------------------------------------
sex                  	 2                   
currentSmoker        	 2                   
BPMeds               	 2                   
prevalentStroke      	 2                   
prevalentHyp         	 2                   
diabetes             	 2                   


The are only 2 distinct values in all the categorical features.

**Let's check for missing values**

In [47]:
print(f"{'Feature': <20} \t {'no. missing values': <20} \t {'Percantage of missing values': <20}")
print('-'*77)
for col in cols:
    print(f"{col: <20} \t {df[col].isna().sum(): <20} \t {np.round(df[col].isna().sum()/df.shape[0],2)*100}%")

Feature              	 no. missing values   	 Percantage of missing values
-----------------------------------------------------------------------------
sex                  	 0                    	 0.0%
age                  	 0                    	 0.0%
education            	 105                  	 2.0%
currentSmoker        	 0                    	 0.0%
cigsPerDay           	 29                   	 1.0%
BPMeds               	 53                   	 1.0%
prevalentStroke      	 0                    	 0.0%
prevalentHyp         	 0                    	 0.0%
diabetes             	 0                    	 0.0%
totChol              	 50                   	 1.0%
sysBP                	 0                    	 0.0%
diaBP                	 0                    	 0.0%
BMI                  	 19                   	 0.0%
heartRate            	 1                    	 0.0%
glucose              	 388                  	 9.0%
TenYearCHD           	 0                    	 0.0%


In [98]:
#Identify and replacing missing values

df[categ_cols] = df[categ_cols].astype(str)

num_cols = df.select_dtypes(include = ['int64', 'float64']).columns
categ_cols = df.select_dtypes(include = ['object']).columns

#We have chosen to fill in the missing numerical and categorical values in the dataset with the 
#most frequently occurring data. This decision is motivated by our belief that the prevalence of 
#heart disease may be linked to the dietary habits of residents based on their geographical 
#location. We are assuming that most patients fall within a similar income bracket and, as a result, 
#are only able to afford certain types of meals.
num_imputer = SimpleImputer(strategy = 'most_frequent')
df[num_cols] = num_imputer.fit_transform(df[num_cols])


In [100]:

cat_imputer = SimpleImputer(strategy = 'most_frequent')
df[categ_cols] = cat_imputer.fit_transform(df[categ_cols])


ValueError: at least one array or dtype is required

In [95]:

print("Missing values:")
print(df.isnull().sum())


Missing values:
Unnamed: 0             0
age                    0
education              0
cigsPerDay             0
totChol                0
sysBP                  0
diaBP                  0
BMI                    0
heartRate              0
glucose                0
TenYearCHD             0
sex_male               0
currentSmoker_Yes      0
BPMeds_Yes             0
prevalentStroke_Yes    0
prevalentHyp_Yes       0
diabetes_Yes           0
dtype: int64


In [63]:
CHD_col = 'TenYearCHD' 
X = df.drop(CHD_col, axis=1)
y = df[CHD_col]

In [64]:
# Dividing the data set into training(60%), validation(20%) and testing(20%)
x_train, x_remaining, y_train, y_remaining = train_test_split(x, y, test_size=0.4, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_remaining, y_remaining, test_size=0.5, random_state=42)

In [65]:
print(f"Training set size: {x_train.shape[0]}")
print(f"Validation set size: {x_val.shape[0]}")
print(f"Testing set size: {x_test.shape[0]}")

Training set size: 2542
Validation set size: 848
Testing set size: 848
